In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import h5py

# Fonctions de base

In [10]:
def file_from_int(n):
    """
    Returns the filename associated to the index of the result

    Parameters
    ----------

    n : int
        Index of the result
    """
    return 'results/simu_' + str(n) + '.h5'

def plot_file(file,  confs=[-3, -2, -1], verbose=True):
    """
    Plots the magnetization and energy per site as a function of time and the last
    configuration

    Parameters
    ----------
    file : int or str
        If int, calls the function `file_from_int` to get the filename. If str, uses this 
        variable as the filename.
    verbose : bool, default = True
        If True, prints the simulation parameters
    """
    if isinstance(file, int) :
        file = file_from_int(file)
        
    g = h5py.File(file, 'r')
    if verbose :
        for key in g.attrs:
            print(key, g.attrs[key])
    
    M=np.array(g['magnetization'])
    E=np.array(g['energy'])
    fig, ax = plt.subplots(2, 3)
    ax[0,0].plot(M)
    ax[0,0].set_xlabel('$t$')
    ax[0,0].set_ylabel('$m$')
    ax[0,1].plot(E)
    ax[0,1].set_xlabel('$t$')
    ax[0,1].set_ylabel('$e$')
    ax[0,2].hist(M, density=True, bins=30)
    ax[0,2].set_xlabel('$m$')
    ax[0,2].set_ylabel('density')
    
    for i, c in enumerate(confs) :
        ax[1,i].imshow(np.array(g['config'])[:, :, c], cmap=plt.cm.bone)
        ax[1,i].set_xlabel('$x$')
        ax[1,i].set_ylabel('$y$')
    fig.set_size_inches(10, 6)
    fig.tight_layout()

def update_results(data=pd.DataFrame()):
    """
    Update the results database by scanning the results file

    Create a database with an entry for each result and with all the attributes of
    the simulation.

    Parameters
    ----------
    data : Pandas DataFrame
        Results database to update. If not given, starts from an empty DataFrame. 
        If given, the files already in the database are not read again.

    Returns
    -------
    data : Pandas DataFrame
        Updated results database
    """
    
    path = 'results/'
    dicts = []
    for d in os.walk(path):
        for f in d[2]:
            if f[:5]=='simu_' and f[-3:]=='.h5' and f[5:-3].isdigit() :
                filename = os.path.join(path, f)

                exists = False
                if not data.empty :
                    if filename in data['file'].values :
                        exists = True


                if not exists :
                    try :
                        g = h5py.File(filename, 'r')
                        mydict = {'file': filename}
                        mydict.update(g.attrs)
                        dicts.append(mydict)
                    except :
                        print('cannot read file ' + filename)

    data = pd.concat((data, pd.DataFrame(dicts)), ignore_index=True)
    
    data.sort_values('file', inplace=True, ignore_index=True)
    return data

def apply_function(fun, col, recompute=False):
    """
    Applies a function to all the result files in the database and updates the database

    Parameters
    ----------
    fun : fun
        Function to apply to the results files: the function takes a simulation file
        as an argument and returns a float.
    col : str
        Name of the field (column) where the result of the function is stored
    recompute : bool, default = False
        Whether the function should be applied to files which already have a number
        in the field `col`.
    """
    for i in range(data.shape[0]) :
        compute = False
        if col in data.columns:
            compute = np.isnan(data.at[i, col]) or recompute
        else:
            compute = True
        
        if compute :
            data.at[i, col] = fun(data.at[i, 'file'])

def apply_function_single(fun, col, file):
    """
    Applies a function to a result file and update the database

    Parameters
    ----------
    fun : fun
        Function to apply to the results files: the function takes a simulation file
        as an argument and returns a float.
    col : str
        Name of the field (column) where the result of the function is stored
    file : int or str
        If int, calls the function `file_from_int` to get the filename. If str, uses this 
        variable as the filename.

    Returns
    -------
    x
        The result of the function
    """
    if isinstance(file, int) :
        file = file_from_int(file)

    i = data[data['file'] == file].index[0]
    x = fun(file)
    data.at[i, col] = x
    return x

def select(kvl):
    """
    Selects the entries with given parameters in the database

    Parameters
    ----------
    kvl : list
        List of key-value pairs

    Returns
    -------
    data : Pandas DataFrame
        Database with only the entries whose key-value pairs correspond to
        the elements of `kvl`.
    """
    
    sel = (data['file'] != None)
    for kv in kvl :
        sel = sel & (data[kv[0]] == kv[1])
    return data[sel]

# Première simulation

Commencez ici !